In [1]:
from result_saver import SaverProvider
provider = SaverProvider()

In [5]:
import numpy as np
# parameter grid
distances = [30, 40, 55]
rounds = np.arange(7, 46, 2) # UNGERADE ZAHLEN!!!
print(rounds)
devices = ["ibm_nazca", "ibm_sherbrooke", "ibm_brisbane", "ibm_cusco"]
logicals = ['0', '1']

[ 7  9 11 13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45]


# Sligthly faster faster triple loop

In [6]:
from tqdm import tqdm

from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit import transpile

from soft_info import find_longest_path_in_hex
from Scratch import metadata_helper


hex_backend = provider.get_backend('ibm_sherbrooke')
path, _, _  = find_longest_path_in_hex(hex_backend)

for nb_of_rounds in tqdm(rounds):
    for distance in (distances):  
        bounded_path = path[:2 * distance - 1]
        layout = bounded_path[1::2] + bounded_path[::2] 
        code = RepetitionCodeCircuit(distance, nb_of_rounds) 
        for device in devices:
            backend = provider.get_backend(device)
            for logical in logicals:          
                qc = code.circuit[logical]                                   
                transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=3)

                n_shots = int(1e6 / distance*nb_of_rounds)
                metadata = metadata_helper(descr='diff rounds v2', code="RepetitionCodeCircuit", distance=distance, rounds=f"{nb_of_rounds}",  logical=logical, layout='_is_hex=True, sherbrooke')
                backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single') # meas_level=1 for IQ  
                


 55%|█████▌    | 11/20 [53:49<52:28, 349.81s/it] 